# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [12]:
symbols = ['AAPL', 'A']
url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={','.join(symbols)}&types=quote&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(url)
print(data)

<Response [200]>


In [20]:
def get_batch(symbols):
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={','.join(symbols)}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    return requests.get(url).json()


## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [17]:
data.json()[symbols[0]]['quote']['peRatio']

42.17

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [18]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [23]:
df = pd.DataFrame(columns=my_columns)

for symbol_list in chunks(stocks['Ticker'], 100):
    data = get_batch(symbol_list)
    for sym in data:
        df = df.append(
            pd.Series(
                [sym,
                 data[sym]['quote']['latestPrice'],
                 data[sym]['quote']['peRatio'],
                 None
                ],
                index=my_columns  
            ),
            ignore_index=True
        )
        

In [24]:
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,123.63,56.13,None
1,AAL,16.38,-1.18,None
2,AAP,159.93,23.66,None
3,AAPL,133.71,40.75,None
4,ABBV,107.82,23.90,None
...,...,...,...,...
500,YUM,113.47,33.10,None
501,ZBH,154.55,-218.07,None
502,ZBRA,394.26,43.88,None
503,ZION,43.45,18.74,None


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [29]:
top = df.sort_values(by='Price-to-Earnings Ratio', ascending=False).head(50)

In [31]:
top

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
225,HPE,12.15,15110.10,None
183,FIS,146.04,2899.40,None
202,GLW,36.00,300.22,None
268,KMI,14.00,295.83,None
296,MAR,134.22,248.82,None
247,IQV,182.75,206.49,None
392,REG,47.32,171.06,None
101,CMG,1392.38,169.11,None
339,NOW,554.78,162.67,None
157,EL,272.64,162,None


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [32]:
def add_shares(df, portfolio_size):
    port_share = portfolio_size/len(df)
    for i in df.index:
        df.loc[i, 'Number of Shares to Buy'] = math.floor(port_share / df.loc[i, 'Price'])
        

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [33]:
add_shares(top, 10000)

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [34]:
top

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
225,HPE,12.15,15110.10,16
183,FIS,146.04,2899.40,1
202,GLW,36.00,300.22,5
268,KMI,14.00,295.83,14
296,MAR,134.22,248.82,1
247,IQV,182.75,206.49,1
392,REG,47.32,171.06,4
101,CMG,1392.38,169.11,0
339,NOW,554.78,162.67,0
157,EL,272.64,162,0


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [38]:
def get_batch(symbols):
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={','.join(symbols)}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    return requests.get(url).json()


In [39]:
data = get_batch(['AAPL'])
print(data['AAPL'])

{'advanced-stats': {'beta': 1.1783494541182586, 'totalCash': 93301138635, 'currentDebt': 106273566277, 'revenue': 283202091060, 'grossProfit': 105500875572, 'totalRevenue': 285980608479, 'EBITDA': 77905957717, 'revenuePerShare': 16.57, 'revenuePerEmployee': 2057948.84, 'debtToEquity': 4.97002900993123, 'profitMargin': 0.21528793378799704, 'enterpriseValue': 2364620633550, 'enterpriseValueToRevenue': 8.4, 'priceToSales': 8.56, 'priceToBook': 34.88, 'forwardPERatio': 34.98544722328614, 'pegRatio': 3.88336456424502, 'peHigh': 42.679238060747686, 'peLow': 17.034687647763242, 'week52highDate': '2020-12-21', 'week52lowDate': '2020-03-12', 'putCallRatio': 0.3246231044278695, 'companyName': 'Apple Inc', 'marketcap': 2283583278144, 'week52high': 137.62, 'week52low': 56.68, 'week52change': 0.8180524442463208, 'sharesOutstanding': 17413521156, 'float': None, 'avg10Volume': 102715692, 'avg30Volume': 107222568, 'day200MovingAvg': 115.4, 'day50MovingAvg': 127.09, 'employees': 139339, 'ttmEPS': 3.29,

In [46]:
#Price-to-earnings ratio
print(data['AAPL']['quote']['peRatio'])

#Price-to-book ratio
print(data['AAPL']['advanced-stats']['priceToBook'])

#Price-to-sales ratio
print(data['AAPL']['advanced-stats']['priceToSales'])

#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
EV = data['AAPL']['advanced-stats']['enterpriseValue']
EBITDA = data['AAPL']['advanced-stats']['EBITDA']
print(EV/EBITDA)

#Enterprise Value divided by Gross Profit (EV/GP)
GP = data['AAPL']['advanced-stats']['grossProfit']
print(EV/GP)




42.11
34.88
8.56
30.35224394698651
22.413279707202467


In [51]:
def parse_response(data, sym):
    pe = data[sym]['quote']['peRatio']
    pb = data[sym]['advanced-stats']['priceToBook']
    ps = data[sym]['advanced-stats']['priceToSales']
    ev = data[sym]['advanced-stats']['enterpriseValue']
    ebitda = data[sym]['advanced-stats']['EBITDA']
    gp = data[sym]['advanced-stats']['grossProfit']
    
    try:
        ev_ebitda = ev/ebitda
    except TypeError:
        ev_ebitda = None
        
    try:
        ev_gp = ev/gp
    except TypeError:
        ev_gp = None
    
    return [pe, pb, ps, ev_ebitda, ev_gp]

In [52]:
parse_response(data, 'AAPL')

[42.11, 34.88, 8.56, 30.35224394698651, 22.413279707202467]

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [80]:

val_cols = ['P/E', 'P/B', 'P/S', 'EV/EBITDA', 'EV/GP']
per_cols = [f"{i} percentile" for i in val_cols]

my_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 'RV'] + val_cols + per_cols

In [81]:
df = pd.DataFrame(columns=my_columns)
df

,Ticker,Price,Number of Shares to Buy,RV,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile


In [82]:
for symbol_list in chunks(stocks['Ticker'], 100):
    data = get_batch(symbol_list)
    for sym in data:
        row = [sym, data[sym]['quote']['latestPrice']]
        row += [None] * 2
        row += parse_response(data, sym)
        row += [None] * len(per_cols)
        df = df.append(
            pd.Series(row, index = my_columns),
            ignore_index = True
        )

In [83]:
df

,Ticker,Price,Number of Shares to Buy,RV,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile
0,A,121.47,None,None,54.88,7.50,7.2300,35.044712,13.814676,None,None,None,None,None
1,AAL,16.09,None,None,-1.13,-1.73,0.4035,-5.902437,1.367286,None,None,None,None,None
2,AAP,162.36,None,None,23.25,2.87,1.0900,10.761817,2.427121,None,None,None,None,None
3,AAPL,133.21,None,None,42.45,35.34,8.3700,29.797495,22.295221,None,None,None,None,None
4,ABBV,107.71,None,None,24.00,12.57,4.8800,16.815216,10.007805,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.44,None,None,32.70,-4.20,6.2100,24.397835,10.366318,None,None,None,None,None
501,ZBH,154.74,None,None,-212.14,2.73,4.6600,26.537081,7.837497,None,None,None,None,None
502,ZBRA,389.77,None,None,45.58,11.02,4.8600,31.481608,11.704674,None,None,None,None,None
503,ZION,44.20,None,None,19.09,1,2.3600,10.607992,2.794160,None,None,None,None,None


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [84]:
df[df[val_cols].isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,RV,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile
40,AON,215.520,None,None,None,None,None,None,None,None,None,None,None,None
71,BRK.B,236.780,None,None,None,None,None,None,None,None,None,None,None,None
118,CTL,11.000,None,None,9.97,None,None,None,None,None,None,None,None,None
136,DISCK,27.270,None,None,None,None,None,None,None,None,None,None,None,None
165,ETFC,50.450,None,None,14.56,None,None,None,None,None,None,None,None,None
168,EVRG,55.930,None,None,None,None,None,None,None,None,None,None,None,None
190,FOX,29.860,None,None,None,None,None,None,None,None,None,None,None,None
192,FRC,153.070,None,None,None,None,None,None,None,None,None,None,None,None
194,FTI,9.400,None,None,None,None,None,None,None,None,None,None,None,None
204,GOOG,1765.350,None,None,None,None,None,None,None,None,None,None,None,None


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
# not necessary

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [72]:
# same as before
df[df[val_cols].isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile
40,AON,220.120,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
71,BRK.B,236.570,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
118,CTL,11.000,None,9.99,None,None,None,None,0.191446,NaN,NaN,NaN,NaN
136,DISCK,27.120,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
165,ETFC,51.040,None,14.67,None,None,None,None,0.274949,NaN,NaN,NaN,NaN
168,EVRG,56.010,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
190,FOX,29.830,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
192,FRC,149.690,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
194,FTI,9.800,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
204,GOOG,1762.970,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [85]:
for val_col, per_col in zip(val_cols, per_cols):
    df[per_col] = df[val_col].rank(pct=True)

In [86]:
df

,Ticker,Price,Number of Shares to Buy,RV,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile
0,A,121.47,None,None,54.88,7.50,7.2300,35.044712,13.814676,0.855397,0.749487,0.800821,0.885010,0.809035
1,AAL,16.09,None,None,-1.13,-1.73,0.4035,-5.902437,1.367286,0.145621,0.047228,0.030801,0.028747,0.055441
2,AAP,162.36,None,None,23.25,2.87,1.0900,10.761817,2.427121,0.458248,0.413758,0.164271,0.256674,0.127310
3,AAPL,133.21,None,None,42.45,35.34,8.3700,29.797495,22.295221,0.782077,0.958932,0.839836,0.845996,0.948665
4,ABBV,107.71,None,None,24.00,12.57,4.8800,16.815216,10.007805,0.472505,0.845996,0.679671,0.540041,0.659138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.44,None,None,32.70,-4.20,6.2100,24.397835,10.366318,0.659878,0.043121,0.756674,0.770021,0.677618
501,ZBH,154.74,None,None,-212.14,2.73,4.6600,26.537081,7.837497,0.020367,0.396304,0.650924,0.802875,0.527721
502,ZBRA,389.77,None,None,45.58,11.02,4.8600,31.481608,11.704674,0.798371,0.825462,0.677618,0.864476,0.726899
503,ZION,44.20,None,None,19.09,1,2.3600,10.607992,2.794160,0.382892,0.099589,0.405544,0.252567,0.151951


In [87]:
# what happened to the NA rows?
df[df[val_cols].isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,RV,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile
40,AON,215.520,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
71,BRK.B,236.780,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
118,CTL,11.000,None,None,9.97,None,None,None,None,0.189409,NaN,NaN,NaN,NaN
136,DISCK,27.270,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
165,ETFC,50.450,None,None,14.56,None,None,None,None,0.270876,NaN,NaN,NaN,NaN
168,EVRG,55.930,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
190,FOX,29.860,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
192,FRC,153.070,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
194,FTI,9.400,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN
204,GOOG,1765.350,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [88]:
for i in df.index:
    mean = 0
    try:
        for col in per_cols:
            mean += df.loc[i, col]
        mean /= len(per_cols)
    except TypeError:
        mean = None
    df.loc[i, 'RV'] = mean

In [89]:
df

,Ticker,Price,Number of Shares to Buy,RV,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile
0,A,121.47,None,0.819950,54.88,7.50,7.2300,35.044712,13.814676,0.855397,0.749487,0.800821,0.885010,0.809035
1,AAL,16.09,None,0.061568,-1.13,-1.73,0.4035,-5.902437,1.367286,0.145621,0.047228,0.030801,0.028747,0.055441
2,AAP,162.36,None,0.284052,23.25,2.87,1.0900,10.761817,2.427121,0.458248,0.413758,0.164271,0.256674,0.127310
3,AAPL,133.21,None,0.875101,42.45,35.34,8.3700,29.797495,22.295221,0.782077,0.958932,0.839836,0.845996,0.948665
4,ABBV,107.71,None,0.639470,24.00,12.57,4.8800,16.815216,10.007805,0.472505,0.845996,0.679671,0.540041,0.659138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.44,None,0.581462,32.70,-4.20,6.2100,24.397835,10.366318,0.659878,0.043121,0.756674,0.770021,0.677618
501,ZBH,154.74,None,0.479638,-212.14,2.73,4.6600,26.537081,7.837497,0.020367,0.396304,0.650924,0.802875,0.527721
502,ZBRA,389.77,None,0.778565,45.58,11.02,4.8600,31.481608,11.704674,0.798371,0.825462,0.677618,0.864476,0.726899
503,ZION,44.20,None,0.258509,19.09,1,2.3600,10.607992,2.794160,0.382892,0.099589,0.405544,0.252567,0.151951


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [90]:
top = df.sort_values(by='RV', ascending=False).head(50)

In [91]:
top

,Ticker,Price,Number of Shares to Buy,RV,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile
339,NOW,572.66,None,0.988528,162.74,44.90,27.78,229.497570,35.388610,0.983707,0.967146,0.995893,0.997947,0.997947
358,PAYC,453.61,None,0.986488,155.04,47.13,34.20,94.158178,36.749588,0.975560,0.971253,0.997947,0.987680,1.000000
13,ADSK,308.32,None,0.973955,161.71,970.00,18.95,104.359899,19.233888,0.979633,1.000000,0.974333,0.991786,0.924025
236,IDXX,513.76,None,0.970926,98.62,161.38,17.53,63.443891,31.666614,0.949084,0.987680,0.967146,0.958932,0.991786
150,DXCM,381.67,None,0.970879,158.28,23.77,20.10,93.529643,28.623880,0.977597,0.928131,0.981520,0.985626,0.981520
31,AMD,94.11,None,0.961453,128.63,29.79,12.79,72.938978,28.411426,0.965377,0.952772,0.936345,0.973306,0.979466
308,MKTX,575.04,None,0.959477,82.26,25.03,34.61,57.449333,34.487660,0.918534,0.934292,1.000000,0.950719,0.993840
345,NVDA,535.70,None,0.957410,85.54,21.21,21.90,64.933649,35.283599,0.926680,0.917864,0.985626,0.960986,0.995893
250,ISRG,846.00,None,0.942599,96.39,10.72,23.44,70.899738,31.550338,0.942974,0.817248,0.993840,0.969199,0.989733
9,ADBE,515.10,None,0.937340,64.92,20.55,19.96,57.135429,22.993951,0.894094,0.907598,0.979466,0.946612,0.958932


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [94]:
add_shares(top, 100000)

In [95]:
top

,Ticker,Price,Number of Shares to Buy,RV,P/E,P/B,P/S,EV/EBITDA,EV/GP,P/E percentile,P/B percentile,P/S percentile,EV/EBITDA percentile,EV/GP percentile
339,NOW,572.66,3,0.988528,162.74,44.90,27.78,229.497570,35.388610,0.983707,0.967146,0.995893,0.997947,0.997947
358,PAYC,453.61,4,0.986488,155.04,47.13,34.20,94.158178,36.749588,0.975560,0.971253,0.997947,0.987680,1.000000
13,ADSK,308.32,6,0.973955,161.71,970.00,18.95,104.359899,19.233888,0.979633,1.000000,0.974333,0.991786,0.924025
236,IDXX,513.76,3,0.970926,98.62,161.38,17.53,63.443891,31.666614,0.949084,0.987680,0.967146,0.958932,0.991786
150,DXCM,381.67,5,0.970879,158.28,23.77,20.10,93.529643,28.623880,0.977597,0.928131,0.981520,0.985626,0.981520
31,AMD,94.11,21,0.961453,128.63,29.79,12.79,72.938978,28.411426,0.965377,0.952772,0.936345,0.973306,0.979466
308,MKTX,575.04,3,0.959477,82.26,25.03,34.61,57.449333,34.487660,0.918534,0.934292,1.000000,0.950719,0.993840
345,NVDA,535.70,3,0.957410,85.54,21.21,21.90,64.933649,35.283599,0.926680,0.917864,0.985626,0.960986,0.995893
250,ISRG,846.00,2,0.942599,96.39,10.72,23.44,70.899738,31.550338,0.942974,0.817248,0.993840,0.969199,0.989733
9,ADBE,515.10,3,0.937340,64.92,20.55,19.96,57.135429,22.993951,0.894094,0.907598,0.979466,0.946612,0.958932


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [96]:
writer = pd.ExcelWriter('Value_trades.xlsx', engine='xlsxwriter')
top.to_excel(writer, 'Recommended trades', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [97]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

ratio_template = writer.book.add_format(
        {
            'num_format':'0.000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [98]:
my_columns

['Ticker',
 'Price',
 'Number of Shares to Buy',
 'RV',
 'P/E',
 'P/B',
 'P/S',
 'EV/EBITDA',
 'EV/GP',
 'P/E percentile',
 'P/B percentile',
 'P/S percentile',
 'EV/EBITDA percentile',
 'EV/GP percentile']

In [99]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Stock Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Robust Value', percent_template], 
    'E': ['Price-to-earnings ratio', percent_template],
    'F': ['Price-to-book ratio', percent_template],
    'G': ['Price-to-sales ratio', percent_template],
    'H': ['Enterprise Value/EBITDA', percent_template], 
    'I': ['Enterprise Value/Gross Profit', percent_template],
    'J': ['Price-to-earnings percentile', percent_template],
    'K': ['Price-to-book percentile', percent_template],
    'L': ['Price-to-sales percentile', percent_template],
    'M': ['Enterprise Value/EBITDA percentile', percent_template], 
    'N': ['Enterprise Value/Gross Profit percentile', percent_template],
}

In [100]:
for column in column_formats:
    writer.sheets['Recommended trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended trades'].write(f"{column}1", column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [101]:
writer.save()